<a href="https://colab.research.google.com/github/sayantika21175/RAG_Projects/blob/main/Rag_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing the required libraries
# Install the pymongo library for MongoDB operations
!pip install pymongo

# Install the langchain library for working with language models and chains
!pip install langchain

# Install the sentence-transformers library for working with sentence embeddings
!pip install sentence-transformers

# Install the langchain-community library for community-contributed langchain extensions
!pip install langchain-community

# Install pymongo with SRV connection string support
!python -m pip install "pymongo[srv]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (4

In [36]:
!python -m pip install "pymongo[srv]"

# Importing the required librariesImporting the required


*   Import the RetrievalQA chain from langchain for question answering over a retrieved set of documents

*   Import the HuggingFaceHub class for using Hugging Face models

* Import the CharacterTextSplitter for splitting text into smaller chunks

* Import the CSVLoader for loading documents from CSV files

* Import the PromptTemplate for creating custom prompts

* Import the os module for operating system interactions



In [21]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
#from langchain.vectorstores import Chroma,FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate
#from langchain.embeddings import HuggingFaceBgeEmbeddings
import os

# Get the HUGGINGFACEHUB_API_TOKEN from userdatato access huggingface models

In [3]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN=userdata.get("hugging_face")

In [4]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=HUGGINGFACEHUB_API_TOKEN

#Loading the dataset

In [7]:
#Unzipping the dataset
!unzip /content/drive/MyDrive/RAG_Project_RecommendationSystem/goodreads_data.csv.zip
#dataset=CSVLoader(file_path=)

Archive:  /content/drive/MyDrive/RAG_Project_RecommendationSystem/goodreads_data.csv.zip
  inflating: goodreads_data.csv      


# Modify the dataset

* considering those rows having nonzero "Genres" value
* Selecting only the columns "Book", "Description", "Genres"

In [8]:
import pandas as pd

df=pd.read_csv("goodreads_data.csv").drop(['Unnamed: 0'],axis=1)
df=df.assign(genre_len=lambda x:len(x["Genres"]))
df=df[df["genre_len"]>0]
df=df[['Book', 'Description','Genres']]
df.to_csv('book_genre.csv',index=False)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df.head()

,Book,Description,Genres
0,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ..."
1,Harry Potter and the Philosopher’s Stone (Harr...,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',..."
2,Pride and Prejudice,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical..."
3,The Diary of a Young Girl,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp..."
4,Animal Farm,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',..."


In [24]:
df.shape

(10000, 3)

In [10]:
df.iloc[5638]

Book                                                 Mockingbird
Description    Mockingbird is a powerful novel of a future wo...
Genres         ['Science Fiction', 'Fiction', 'Dystopia', 'Po...
Name: 5638, dtype: object

#load the dataset using CSVLoader

* load a CSV file into a list of Documents
* Here each dataset contains one row of the CSV file and metadata
* metadata contains the row number

https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.csv_loader.CSVLoader.html

In [11]:
loader=CSVLoader("book_genre.csv")

In [12]:
dataset=loader.load()
#dataset

In [13]:
dataset[0]

Document(metadata={'source': 'book_genre.csv', 'row': 0}, page_content='Book: To Kill a Mockingbird\nDescription: The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.\nGenres: [\'Classics\', \'Fiction\', \'Historical Fiction\', \'Scho

# Spliting the text into chunks
 * each chunk contains 1000 characters with 100 overlapping character

In [14]:
text_spliter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
texts=text_spliter.split_documents(dataset)


In [22]:
text_splitter_rec=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
texts_rec=text_splitter_rec.split_documents(dataset)

In [23]:
len(texts_rec)

23601

In [25]:
texts_rec[0]

Document(metadata={'source': 'book_genre.csv', 'row': 0}, page_content='Book: To Kill a Mockingbird\nDescription: The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.\nGenres: [\'Classics\', \'Fiction\', \'Historical Fiction\', \'Scho

In [15]:
len(texts)

10045

In [20]:
texts[0]

Document(metadata={'source': 'book_genre.csv', 'row': 0}, page_content='Book: To Kill a Mockingbird\nDescription: The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.\nGenres: [\'Classics\', \'Fiction\', \'Historical Fiction\', \'Scho

# importing sentence_transformers for embedding generation

In [26]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#Embedding dim=384
max token=512

* all-MiniLM-L12-v2 embedding model is used here

In [27]:
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
!curl ipecho.net/plain

34.87.162.246

In [38]:
import pymongo
# from google.colab import userdata

# Connecting to mongodb atlas vector database

In [39]:
from pymongo import MongoClient

In [40]:
def get_mongo_client(mongo_uri):
  try:
    client=pymongo.MongoClient(mongo_uri)
    print("Successfully connected to mongo db")
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f"Connection failed {e}")
    return None

In [41]:
mongo_uri=userdata.get("mongo_uri")
if not mongo_uri:
  print("mongo_uri has not been set as environment variable")
mongo_client=get_mongo_client(mongo_uri)


Successfully connected to mongo db


# Creating an embedding function and generate embeddings of the page_content of each row of the dataset

* with a batch size of 100

In [42]:
#embed_model= SentenceTransformer(model,device='cuda')

def Embed(text_batch):
    # We manually encode using sentence_transformer since LangChain
    # HuggingfaceEmbeddings does not support specifying a batch size yet.

    embeddings = embed_model.encode(
        text_batch,
        batch_size=100, # Large batch size to maximize GPU utilization.
        device="cuda",
    ).tolist()

    #return list(zip(text_batch, embeddings))
    return embeddings

In [43]:
import time

In [44]:
texts_rec[1].page_content

"Book: Harry Potter and the Philosopher’s Stone (Harry Potter, #1)\nDescription: Harry Potter thinks he is an ordinary boy - until he is rescued by an owl, taken to Hogwarts School of Witchcraft and Wizardry, learns to play Quidditch and does battle in a deadly duel. The Reason ... HARRY POTTER IS A WIZARD!\nGenres: ['Fantasy', 'Fiction', 'Young Adult', 'Magic', 'Childrens', 'Middle Grade', 'Classics']"

In [45]:
start_time=time.time()
print(f"start time is {start_time}")
documents_list=[]
batch_no=0


for text_batch in texts_rec:
  # print(text_batch.page_content)
  # print(text_batch.metadata)
  # print(len(text_batch.page_content))
  #embed_output_doc.append(Embed(text_batch.page_content))
  embeddings=Embed(text_batch.page_content)
  document = {
        "source": text_batch.metadata['source'],
        "row": text_batch.metadata['row'],
        "page_content": text_batch.page_content,
        "embedding": embeddings
    }
  documents_list.append(document)
  end_time=time.time()
  batch_no=batch_no+1
  print(f"for batch no {batch_no} takes {end_time}")

end_time_final=time.time()
print(f"the total time is {end_time_final-start_time}")



Streaming output truncated to the last 5000 lines.
for batch no 18603 takes 1721063359.5202699
for batch no 18604 takes 1721063359.531833
for batch no 18605 takes 1721063359.5425005
for batch no 18606 takes 1721063359.5543277
for batch no 18607 takes 1721063359.5656
for batch no 18608 takes 1721063359.577121
for batch no 18609 takes 1721063359.5888536
for batch no 18610 takes 1721063359.5996258
for batch no 18611 takes 1721063359.6120875
for batch no 18612 takes 1721063359.623557
for batch no 18613 takes 1721063359.6339462
for batch no 18614 takes 1721063359.644342
for batch no 18615 takes 1721063359.6567802
for batch no 18616 takes 1721063359.671309
for batch no 18617 takes 1721063359.6852856
for batch no 18618 takes 1721063359.696862
for batch no 18619 takes 1721063359.707333
for batch no 18620 takes 1721063359.721166
for batch no 18621 takes 1721063359.7323344
for batch no 18622 takes 1721063359.742994
for batch no 18623 takes 1721063359.7545068
for batch no 18624 takes 1721063359.7

In [ ]:
len(documents_list)

10045

In [ ]:
#embed_output_doc[0]

In [ ]:
#embed_output_doc[0]

# check if the collection(book_set) in "book" database in mongodb has any document or not. if yes delete those documents

In [ ]:
#to ensure there is not any data
db=mongo_client["book"]
collection=db["book_set"]
#collection.delete_many({})

In [ ]:
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000196'), 'opTime': {'ts': Timestamp(1720630717, 18), 't': 406}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1720630717, 18), 'signature': {'hash': b'%Q&\x86h\x8e\x16\xcaF\xaa\xd6\xd4\xc3\x11\xbad\xd0\xee\xbf\x88', 'keyId': 7348106048890208268}}, 'operationTime': Timestamp(1720630717, 18)}, acknowledged=True)

In [ ]:
documents_list[0]

{'source': 'book_genre.csv',
 'row': 0,
 'page_content': 'Book: To Kill a Mockingbird\nDescription: The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.\nGenres: [\'Classics\', \'Fiction\', \'Historical Fiction\', \'School\', \'Litera

In [ ]:
#documents[0]

#storing the dataset or documents in mongodb atlas vector database

In [ ]:
start_ingestion_time=time.time()
collection.insert_many(documents_list)
end_ingestion_time=time.time()
print(f"document ingestion is done and it took {end_ingestion_time-start_ingestion_time}")


document ingestion is done and it took 17.16020393371582


# Query the database to fetch similar documents and pass the result to llm for further processing.

* Generate embeddings of the query
* Do semantic search in vector database with that query embeddings
* get the similar documents
* pass those similar documents to the llm with the query
* llm finds out the most relevent one to the query
* have used flan-t5-base as llm

In [ ]:
query="Which books except 'To Kill A Mocking Bird' are similar to it?"

In [ ]:
embed_query=embed_model.encode(query).tolist()
# embed_query

In [ ]:
#embed_query_1

In [ ]:
result=collection.aggregate([
    {"$vectorSearch":{
            "queryVector":embed_query,
            "path":"embedding",
            "numCandidates":1000,
            "limit":5,
            "index":"vector_index"
        }
    }
])

In [ ]:
result

In [ ]:
def formated_doc(doc):
  #return "\n\n".join(doc.get('page_content'))
  return

In [ ]:
formated_doc_list=[]
for doc in result:
  print(doc)
  formated_doc_list.append(doc.get('page_content'))

  #print(doc)

{'_id': ObjectId('668ec431ced0c359cae0345c'), 'source': 'book_genre.csv', 'row': 7685, 'page_content': 'Book: A Guide To To Kill A Mockingbird\nDescription: TheLiterature Made Easy Series is more than just plot summaries. Each book describes a classic novel and drama by explaining themes, elaborating on characters, and discussing each author\'s unique literary style, use of language, and point of view. Extensive illustrations and imaginative, enlightening use of graphics help to make each book in this series livelier, easier, and more fun to use than ordinary literature plot summaries. An unusual feature, "Mind Map" is a diagram that summarizes and interrelates the most important details that students need to understand about a given work. Appropriate for middle and high school students.\nGenres: []', 'embedding': [0.017193742096424103, -0.006330861244350672, -0.027450820431113243, 0.026722129434347153, 0.011244053021073341, 0.05652952939271927, -0.027344802394509315, -0.01534633804112

In [ ]:
formated_doc_list

['Book: A Guide To To Kill A Mockingbird\nDescription: TheLiterature Made Easy Series is more than just plot summaries. Each book describes a classic novel and drama by explaining themes, elaborating on characters, and discussing each author\'s unique literary style, use of language, and point of view. Extensive illustrations and imaginative, enlightening use of graphics help to make each book in this series livelier, easier, and more fun to use than ordinary literature plot summaries. An unusual feature, "Mind Map" is a diagram that summarizes and interrelates the most important details that students need to understand about a given work. Appropriate for middle and high school students.\nGenres: []',
 'Book: Harper Lee\'s To Kill a Mockingbird\nDescription: Published in 1960 and awarded the Pulitzer Prize in 1961, To Kill a Mockingbird is required reading for many middle and high school students. The coming-of-age tale of its young narrator, Jean Louise "Scout" Finch, of Maycomb, Alab

In [ ]:
prompt_template = """
Form a grammatically correct sentence as a response.
Compare the book given in question with others in the retriever based on genre and description.
Return a complete sentence with the full title of the book and describe the similarities between the books.
question: {question}
context: {context}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [ ]:
llm = HuggingFaceHub(repo_id = "google/flan-t5-base", model_kwargs={"temperature":0.6,"max_length": 1000})


* Here if we only pass the query to the llm it hallucinates

In [ ]:
llm("Which books except 'To Kill A Mocking Bird' are similar to it?")

'samuel sandler and samuel sandler'

In [ ]:
from langchain.chains import LLMChain

In [ ]:
qa_chain = LLMChain(llm=llm,prompt=prompt)

# if we pass the proper context to the llm with query it provides the answer

In [ ]:
qa_chain.run({'question': query, 'context': formated_doc_list})

"Go Set a Watchman is a companion to Harper Lee's To Kill a Mockingbird."